In [8]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as mat
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from keras import layers, optimizers, models
from sklearn.preprocessing import LabelEncoder

In [9]:
#讀取資料並做資料前處理將特殊字元處理掉
train_data =pd.read_csv('adult.data',sep=" ", header=None)
train_data = train_data.replace({'\$': '', ',': '','\.':'','<=50K':'1','>50K':'0'}, regex=True) #砍掉換行&逗號
train_data.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','label']

test_data =pd.read_csv('adult.test',sep=" ", header=None)
test_data = test_data.replace({'\$': '', ',': '','<=50K.':'1','>50K.':'0'}, regex=True) #砍掉換行&逗號
test_data.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','label']


In [10]:
#合併train data和test data
data_1 = train_data
data_1=data_1.append(test_data)
#data_1

In [11]:
#將缺值欄位補上平均值
data_1.replace('?', np.nan, inplace=True)
data_1=data_1.fillna(data_1.mean())
data_1 = data_1.apply(lambda x:x.fillna(x.value_counts().index[0]))
#data_1

In [12]:
#將資料轉成int
data_1[['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week','label'] ]=data_1[['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week','label']].astype(str).astype(int)
#將文字資料(要做one hot encoding和label切出)
data_cat = data_1[['workclass','education','marital-status','occupation','relationship','race','sex','native-country']]
data_hours = data_1[['hours-per-week']]

In [13]:
#one hot encoding
data_cat = pd.get_dummies(data_cat)
#data_cat

In [14]:
#newdata = data_1.drop(['hours-per-week','workclass','education','marital-status','occupation','relationship','race','sex','native-country'],axis=1).join(data_cat,how='left')
#newdata

In [15]:
#將原始資料合併one hot encoding後的資料
newdata1 = data_1.drop(['workclass','education','marital-status','occupation','relationship','race','sex','native-country'],axis=1)
newdata_merge = pd.concat([newdata1,data_cat],axis=1).reindex(data_1.index)

In [16]:
#newdata_merge

In [17]:
#將資料正規化
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(newdata_merge)
data_norm= pd.DataFrame(np_scaled, columns = newdata_merge.columns)
data_norm.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,label,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0.301370,0.044131,0.800000,0.02174,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.452055,0.048052,0.800000,0.00000,0.0,0.122449,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.287671,0.137581,0.533333,0.00000,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.493151,0.150486,0.400000,0.00000,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.150685,0.220635,0.800000,0.00000,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#test_data=newdata_merge.iloc[32561:]
#test_data

In [19]:
#train_data=newdata_merge.iloc[:32561]
#train_data

In [20]:
test_data1=data_norm.iloc[32561:] #分解train data和test data
test_data1

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,label,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
32561,0.109589,0.145129,0.400000,0.000000,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32562,0.287671,0.052451,0.533333,0.000000,0.0,0.500000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32563,0.150685,0.219649,0.733333,0.000000,0.0,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32564,0.369863,0.100153,0.600000,0.076881,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32565,0.013699,0.061708,0.600000,0.000000,0.0,0.295918,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.301370,0.137428,0.800000,0.000000,0.0,0.357143,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48838,0.643836,0.209130,0.533333,0.000000,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48839,0.287671,0.245379,0.800000,0.000000,0.0,0.500000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48840,0.369863,0.048444,0.800000,0.054551,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [21]:
train_data1=data_norm.iloc[:32561] #分解train data和test data
train_data1

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,label,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0.301370,0.044131,0.800000,0.021740,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.452055,0.048052,0.800000,0.000000,0.0,0.122449,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.287671,0.137581,0.533333,0.000000,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.493151,0.150486,0.400000,0.000000,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.150685,0.220635,0.800000,0.000000,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.136986,0.165763,0.733333,0.000000,0.0,0.377551,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32557,0.315068,0.096129,0.533333,0.000000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32558,0.561644,0.094462,0.533333,0.000000,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32559,0.068493,0.128004,0.533333,0.000000,0.0,0.193878,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
train_true=data_hours.iloc[:32561] #擷取traindata label
#train_true

In [23]:
test_true=data_hours.iloc[32561:] #擷取testdata label
#test_true

In [24]:
from keras import layers, optimizers, models
from sklearn.preprocessing import LabelEncoder

X_train1 = train_data1.drop('hours-per-week', axis=1)
y_train1 = train_true

X_test1 = test_data1.drop('hours-per-week', axis=1)
y_test1 = test_true

In [25]:
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers.core import Dense , Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping

#dense指最普通的全連接層型態
#dropout減少節點 通常設0.2~0.5 最高不會超過0.5
#2層(最後一層ouput不算)


early_stopping = EarlyStopping(monitor='val_loss', patience=200,restore_best_weights=True)
model = models.Sequential()
model.add(layers.Dense(16, input_shape=(X_train1.shape[1],), activation="relu")) 
model.add(layers.Dropout(0.30)) #d砍上一層的節點
model.add(layers.Dense(8, activation="relu"))
model.add(layers.Dropout(0.30))
model.add(layers.Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train1, y_train1, validation_split=0.2, epochs=1000000, batch_size=128,callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test1, y_test1)
print("Test Acc : " + str(accuracy))
print("Test Loss : " + str(loss))
y_pred1 = model.predict(X_test1)
print('MSE為：',mean_squared_error(y_test1,y_pred1))
#print('MSE為(直接计算)：',np.mean((y_test-y_pred)**2))
print('RMSE為：',np.sqrt(mean_squared_error(y_test1,y_pred1)))

Train on 26048 samples, validate on 6513 samples
Epoch 1/1000000
26048/26048 [==============================] - 0s 16us/step - loss: 1198.4975 - accuracy: 0.0046 - val_loss: 229.1376 - val_accuracy: 0.0123
Epoch 2/1000000
26048/26048 [==============================] - 0s 10us/step - loss: 335.6535 - accuracy: 0.0204 - val_loss: 156.8854 - val_accuracy: 0.0613
Epoch 3/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 300.7744 - accuracy: 0.0238 - val_loss: 149.0055 - val_accuracy: 0.0649
Epoch 4/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 283.9771 - accuracy: 0.0230 - val_loss: 145.1805 - val_accuracy: 0.0680
Epoch 5/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 277.4481 - accuracy: 0.0235 - val_loss: 143.3571 - val_accuracy: 0.0665
Epoch 6/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 274.4005 - accuracy: 0.0249 - val_loss: 144.0553 - val_accuracy: 0.0536
Epoch 7/1000000
26

26048/26048 [==============================] - 0s 9us/step - loss: 200.5586 - accuracy: 0.0324 - val_loss: 136.3280 - val_accuracy: 0.0510
Epoch 53/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 196.9878 - accuracy: 0.0332 - val_loss: 133.3950 - val_accuracy: 0.0520
Epoch 54/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 199.5545 - accuracy: 0.0319 - val_loss: 135.5194 - val_accuracy: 0.0491
Epoch 55/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 195.9690 - accuracy: 0.0324 - val_loss: 135.8372 - val_accuracy: 0.0485
Epoch 56/1000000
26048/26048 [==============================] - 0s 10us/step - loss: 197.1600 - accuracy: 0.0347 - val_loss: 130.5084 - val_accuracy: 0.0608
Epoch 57/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 195.2989 - accuracy: 0.0337 - val_loss: 134.4365 - val_accuracy: 0.0488
Epoch 58/1000000
26048/26048 [==============================] - 0s 9us/step - lo

26048/26048 [==============================] - 0s 9us/step - loss: 164.5974 - accuracy: 0.0352 - val_loss: 129.6258 - val_accuracy: 0.0622
Epoch 105/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 161.6993 - accuracy: 0.0371 - val_loss: 127.4297 - val_accuracy: 0.0628
Epoch 106/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 162.8197 - accuracy: 0.0352 - val_loss: 129.2022 - val_accuracy: 0.0636
Epoch 107/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 162.7708 - accuracy: 0.0368 - val_loss: 129.4205 - val_accuracy: 0.0639
Epoch 108/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 163.7750 - accuracy: 0.0382 - val_loss: 129.2687 - val_accuracy: 0.0628
Epoch 109/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 161.1259 - accuracy: 0.0409 - val_loss: 129.5774 - val_accuracy: 0.0582
Epoch 110/1000000
26048/26048 [==============================] - 0s 9us/step

Epoch 156/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 141.2545 - accuracy: 0.0472 - val_loss: 126.9296 - val_accuracy: 0.0640
Epoch 157/1000000
26048/26048 [==============================] - 0s 10us/step - loss: 141.6666 - accuracy: 0.0446 - val_loss: 129.2994 - val_accuracy: 0.0622
Epoch 158/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 141.5836 - accuracy: 0.0422 - val_loss: 127.3409 - val_accuracy: 0.0637
Epoch 159/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 140.6122 - accuracy: 0.0457 - val_loss: 128.8795 - val_accuracy: 0.0631
Epoch 160/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 141.8453 - accuracy: 0.0453 - val_loss: 127.8351 - val_accuracy: 0.0669
Epoch 161/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 141.7796 - accuracy: 0.0448 - val_loss: 128.4215 - val_accuracy: 0.0626
Epoch 162/1000000
26048/26048 [==========================

26048/26048 [==============================] - 0s 11us/step - loss: 130.9961 - accuracy: 0.0501 - val_loss: 128.2222 - val_accuracy: 0.0645
Epoch 208/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 131.3454 - accuracy: 0.0499 - val_loss: 127.2332 - val_accuracy: 0.0682
Epoch 209/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 131.0661 - accuracy: 0.0517 - val_loss: 126.8651 - val_accuracy: 0.0654
Epoch 210/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 131.6114 - accuracy: 0.0487 - val_loss: 126.9881 - val_accuracy: 0.0671
Epoch 211/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 130.9881 - accuracy: 0.0517 - val_loss: 126.7759 - val_accuracy: 0.0643
Epoch 212/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 130.7196 - accuracy: 0.0500 - val_loss: 126.6330 - val_accuracy: 0.0633
Epoch 213/1000000
26048/26048 [==============================] - 0s 9us/ste

26048/26048 [==============================] - 0s 9us/step - loss: 127.4189 - accuracy: 0.0569 - val_loss: 127.0886 - val_accuracy: 0.0648
Epoch 260/1000000
26048/26048 [==============================] - 0s 16us/step - loss: 126.5966 - accuracy: 0.0565 - val_loss: 127.4467 - val_accuracy: 0.0645
Epoch 261/1000000
26048/26048 [==============================] - 0s 15us/step - loss: 127.0585 - accuracy: 0.0531 - val_loss: 127.2424 - val_accuracy: 0.0671
Epoch 262/1000000
26048/26048 [==============================] - 0s 13us/step - loss: 126.9312 - accuracy: 0.0540 - val_loss: 127.7742 - val_accuracy: 0.0656
Epoch 263/1000000
26048/26048 [==============================] - 0s 14us/step - loss: 126.7684 - accuracy: 0.0550 - val_loss: 127.0559 - val_accuracy: 0.0663
Epoch 264/1000000
26048/26048 [==============================] - 0s 14us/step - loss: 127.1887 - accuracy: 0.0537 - val_loss: 127.2190 - val_accuracy: 0.0645
Epoch 265/1000000
26048/26048 [==============================] - 0s 14u

26048/26048 [==============================] - 0s 9us/step - loss: 126.2453 - accuracy: 0.0594 - val_loss: 128.3805 - val_accuracy: 0.0697
Epoch 312/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 126.9143 - accuracy: 0.0586 - val_loss: 127.8728 - val_accuracy: 0.0669
Epoch 313/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 126.8051 - accuracy: 0.0607 - val_loss: 127.7386 - val_accuracy: 0.0648
Epoch 314/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 126.6560 - accuracy: 0.0586 - val_loss: 127.8776 - val_accuracy: 0.0654
Epoch 315/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 127.1891 - accuracy: 0.0580 - val_loss: 127.8983 - val_accuracy: 0.0685
Epoch 316/1000000
26048/26048 [==============================] - 0s 9us/step - loss: 127.0075 - accuracy: 0.0584 - val_loss: 128.5138 - val_accuracy: 0.0689
Epoch 317/1000000
26048/26048 [==============================] - 0s 9us/step

26048/26048 [==============================] - 0s 11us/step - loss: 126.8184 - accuracy: 0.0608 - val_loss: 128.1006 - val_accuracy: 0.0677
Epoch 364/1000000
26048/26048 [==============================] - 0s 10us/step - loss: 126.4443 - accuracy: 0.0593 - val_loss: 128.0670 - val_accuracy: 0.0674
Epoch 365/1000000
26048/26048 [==============================] - 0s 10us/step - loss: 126.2172 - accuracy: 0.0574 - val_loss: 127.7651 - val_accuracy: 0.0686
Epoch 366/1000000
26048/26048 [==============================] - 0s 11us/step - loss: 126.6970 - accuracy: 0.0572 - val_loss: 128.0962 - val_accuracy: 0.0666
Epoch 367/1000000
26048/26048 [==============================] - 0s 11us/step - loss: 126.8855 - accuracy: 0.0570 - val_loss: 128.0892 - val_accuracy: 0.0662
Epoch 368/1000000
26048/26048 [==============================] - 0s 10us/step - loss: 126.7864 - accuracy: 0.0593 - val_loss: 128.1999 - val_accuracy: 0.0692
Epoch 369/1000000
26048/26048 [==============================] - 0s 11

In [28]:
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers.core import Dense , Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping

#dense指最普通的全連接層型態
#dropout減少節點 通常設0.2~0.5 最高不會超過0.5
#2層(最後一層ouput不算)


early_stopping = EarlyStopping(monitor='val_loss', patience=200,restore_best_weights=True)
model = models.Sequential()
model.add(layers.Dense(64, input_shape=(X_train.shape[1],), activation="relu")) 
model.add(layers.Dropout(0.30)) #d砍上一層的節點
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dropout(0.30))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dropout(0.30))
model.add(layers.Dense(8, activation="relu"))
model.add(layers.Dropout(0.30))
model.add(layers.Dense(1))

model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000000, batch_size=128,callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Acc : " + str(accuracy))
print("Test Loss : " + str(loss))
y_pred = model.predict(X_test)
print('MSE為：',mean_squared_error(y_test,y_pred))
#print('MSE為(直接计算)：',np.mean((y_test-y_pred)**2))
print('RMSE為：',np.sqrt(mean_squared_error(y_test,y_pred)))

NameError: name 'X_train' is not defined